# Coursework 2 - Associated Rule Mining

Referencces
- https://towardsdatascience.com/apriori-association-rule-mining-explanation-and-python-implementation-290b42afdfc6
- asdasd


**Apriori algorithm**- All non-empty subsets of a frequent itemset must also be frequent.

Reference: Module of class<br>
Fk: frequent k-itemsets
<br>Lk: candidate k-itemsets

Let k=1
<br>Generate F1 = {frequent 1-itemsets}
<br>Repeat until Fk is empty
- Candidate Joining: Generate Lk+1 from Fk
- Candidate Pruning: Prune candidate itemsets in Lk+1 containing subsets of length k that are infrequent 
- Support Counting: Count the support of each candidate in Lk+1 by scanning the DB
- Candidate Elimination: Eliminate candidates in Lk+1 that are infrequent, leaving only those that are frequent => Fk+1

#### 1(a)Importing libraries

In [21]:
import csv
from  csv import reader
from collections import defaultdict
from itertools import chain, combinations
import numpy as np
import pandas as pd

- Imported Libraries that will be used for executing the program.

Reference 2
- https://docs.python.org/3/library/stdtypes.html
- Frozen set-->The frozenset type is immutable and hashable — its contents cannot be altered after it is created; it can therefore be used as a dictionary key or as an element of another set. https://docs.python.org/3/library/stdtypes.html#:~:text=in%20version%203.3.-,Set%20Types,-%E2%80%94%20set%2C%20frozenset%C2%B6
    

### Writing data in different chunksize in files

In [22]:
# chunk_size=50000
# batch_no=1
# for chunk in pd.read_csv('OnlineRetail.csv',chunksize=chunk_size):
#     chunk.to_csv('chunk'+str(batch_no)+'.csv',index=False)
#     batch_no+=1

# The code was written to segregate the orignal dataset to smaller dataset files So, we do not end up with memory allocation error.

#### Loading File

In [23]:
# print(pd.options.display.max_rows)
# itemSet, itemSets = get_from
data = []
# with open('drive/MyDrive/datasets/GroceryStore.csv', newline='') as f:
# Was Loading the file from Google drive then shifted to local Repository 
with open('GroceryStore.csv', newline='') as f:
    for ele in csv.reader(f):       
        data.append(list(ele[0].split(",")))
        # dataset rows element is split into sections using comma as a delimiter
data

[['MILK', 'BREAD', 'BISCUIT'],
 ['BREAD', 'MILK', 'BISCUIT', 'CORNFLAKES'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['JAM', 'MAGGI', 'BREAD', 'MILK'],
 ['MAGGI', 'TEA', 'BISCUIT'],
 ['BREAD', 'TEA', 'BOURNVITA'],
 ['MAGGI', 'TEA', 'CORNFLAKES'],
 ['MAGGI', 'BREAD', 'TEA', 'BISCUIT'],
 ['JAM', 'MAGGI', 'BREAD', 'TEA'],
 ['BREAD', 'MILK'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'COCK', 'BISCUIT', 'CORNFLAKES'],
 ['COFFEE', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'COCK'],
 ['BREAD', 'SUGER', 'BISCUIT'],
 ['COFFEE', 'SUGER', 'CORNFLAKES'],
 ['BREAD', 'SUGER', 'BOURNVITA'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['BREAD', 'COFFEE', 'SUGER'],
 ['TEA', 'MILK', 'COFFEE', 'CORNFLAKES']]

In [24]:
dataset = data # making a copy of data

#### Unique elements in dataset
Reference:
- https://japp.io/machine-learning/apriori-algorithm-program-in-python-from-scratch/

In [25]:
# data = [
#         ['T100',['I1','I2','I5']],
#         ['T200',['I2','I4']],
#         ['T300',['I2','I3']],
#         ['T400',['I1','I2','I4']],
#         ['T500',['I1','I3']],
#         ['T600',['I2','I3']],
#         ['T700',['I1','I3']],
#         ['T800',['I1','I2','I3','I5']],
#         ['T900',['I1','I2','I3']]
#         ]

''' 
Finding out the unique elemnts from every set given in the dataset.
Result is stored in a list
'''

init = []
for i in data:
    for q in i:
        if(q not in init):
            init.append(q)
init = sorted(init)
print(init)
print("Length",len(init))

['BISCUIT', 'BOURNVITA', 'BREAD', 'COCK', 'COFFEE', 'CORNFLAKES', 'JAM', 'MAGGI', 'MILK', 'SUGER', 'TEA']
Length 11


#### Support

In [26]:
# Choosing support as 1
# sp = 0.5
# s = int(sp*len(init))
s=1
print(s)

1


### 1(b)(c)Frequent Item set & Apriori Algorithm brute force

In [27]:
print(len(data)) # data length analyzed

20


frequency of occurence

In [28]:
''' 
Finding out the frequency of occurence of each item throughout the dataset.
'''

from collections import Counter
c = Counter() 

''' 
for each item in the frequency list we create a dictionary with key as a item and value as the frequency of the item in dataset.
'''

for i in init:
    for j in range(0,len(data)): 
        if(i in data[j]):
            c[i]+=1
print("C1:")
for i in c:
    print(str([i])+": "+str(c[i]))
print()


C1:
['BISCUIT']: 7
['BOURNVITA']: 4
['BREAD']: 13
['COCK']: 3
['COFFEE']: 8
['CORNFLAKES']: 6
['JAM']: 2
['MAGGI']: 5
['MILK']: 5
['SUGER']: 6
['TEA']: 7



### Possible Itemset 
- https://japp.io/machine-learning/apriori-algorithm-program-in-python-from-scratch/ 

In [29]:
'''

All possible subsets are being made

'''


l = Counter()
for i in c: # for every elemnt in (c) dictonary we are creating the likelihood table based on support.
    if(c[i] >= s):
        l[frozenset([i])]+=c[i]
print("L1:")
for i in l: # All elements of likelihood table are being printed
    print(str(list(i))+": "+str(l[i]))
print() 

'''
Creating n-itemsets 
'''
pl = l
pos = 1
for count in range (2,100000000): # Running the loop for maximum number of entries
    nc = set()
    temp = list(l)
    for i in range(0,len(temp)): # for complete length
        for j in range(i+1,len(temp)):
            t = temp[i].union(temp[j])
            if(len(t) == count):
                nc.add(temp[i].union(temp[j]))
    nc = list(nc)
    c = Counter()
    for i in nc:
        c[i] = 0
        for q in data:
####################################            
            temp = set(q)
            if(i.issubset(temp)):
                c[i]+=1
    print("C"+str(count)+":")
    for i in c:
        print(str(list(i))+": "+str(c[i]))
    print()
    l = Counter() 
    for i in c:
        if(c[i] >= s):
            l[i]+=c[i]
    print("L"+str(count)+":")
    for i in l:
        print(str(list(i))+": "+str(l[i]))
    print()
    if(len(l) == 0):
        break
    pl = l
    pos = count
print("Result: ")
print("L"+str(pos)+":")
for i in pl:
    print(str(list(i))+": "+str(pl[i])) # pl[i] is the frequency of that particular itemset
print()

L1:
['BISCUIT']: 7
['BOURNVITA']: 4
['BREAD']: 13
['COCK']: 3
['COFFEE']: 8
['CORNFLAKES']: 6
['JAM']: 2
['MAGGI']: 5
['MILK']: 5
['SUGER']: 6
['TEA']: 7

C2:
['MAGGI', 'BOURNVITA']: 0
['BREAD', 'SUGER']: 4
['COCK', 'JAM']: 0
['MAGGI', 'MILK']: 1
['TEA', 'MAGGI']: 4
['COCK', 'BOURNVITA']: 0
['MAGGI', 'BISCUIT']: 2
['CORNFLAKES', 'JAM']: 0
['BREAD', 'COFFEE']: 3
['BOURNVITA', 'JAM']: 0
['SUGER', 'CORNFLAKES']: 1
['MAGGI', 'SUGER']: 0
['BREAD', 'MILK']: 4
['TEA', 'COFFEE']: 1
['SUGER', 'COCK']: 0
['COCK', 'CORNFLAKES']: 2
['MILK', 'BISCUIT']: 2
['COFFEE', 'MAGGI']: 0
['TEA', 'COCK']: 0
['MAGGI', 'JAM']: 2
['BOURNVITA', 'CORNFLAKES']: 0
['COFFEE', 'BISCUIT']: 2
['SUGER', 'BOURNVITA']: 2
['BREAD', 'MAGGI']: 3
['BOURNVITA', 'MILK']: 0
['TEA', 'CORNFLAKES']: 2
['COCK', 'BISCUIT']: 2
['TEA', 'BISCUIT']: 2
['COFFEE', 'MILK']: 1
['SUGER', 'BISCUIT']: 1
['COFFEE', 'BOURNVITA']: 1
['BREAD', 'JAM']: 2
['COFFEE', 'CORNFLAKES']: 4
['MAGGI', 'CORNFLAKES']: 1
['COFFEE', 'JAM']: 0
['SUGER', 'MILK']: 0


1(d). Association rules for the subsets.
- https://japp.io/machine-learning/apriori-algorithm-program-in-python-from-scratch/ 

- https://www.geeksforgeeks.org/python-subtraction-of-dictionaries/ 

-



In [30]:
'''
Finding out the probability of association in between the sets
'''
from itertools import combinations
for l in pl:
    c = [frozenset(q) for q in combinations(l,len(l)-1)]
    mmax = 0
    '''
    a Element of dictionary c
    b It is the result of subtraction of values of the respective key
    c Frozen set combination of pl
    ab 
    l is the likelihood dictonary
    pl is the dictonary for n - itemset
    '''
    for a in c: 
        b = l-a
        ab = l
        sab = 0 # Intializing to zero
        sa = 0 # Intializing to zero
        sb = 0 # Intializing to zero
        for q in data:
####################################    
            '''
            Finding the element or subset in rest of the subsets
            '''
            temp = set(q)
            if(a.issubset(temp)):
                sa+=1
            if(b.issubset(temp)):
                sb+=1
            if(ab.issubset(temp)):
                sab+=1
        if(sa == 0):
            temp = 0
        else:
            temp = sab/sa*100
        if(temp > mmax):  # max sup number
            '''
            Finding out the maximum support number
            '''
            mmax = temp
        if(sb == 0):
            temp = 0
        else:
            temp = sab/sb*100
        if(temp > mmax):
            mmax = temp
        if(sa == 0 or sb == 0):
            print(str(list(a))+" -> "+str(list(b))+" = "+str(0)+"%")
            print(str(list(b))+" -> "+str(list(a))+" = "+str(0)+"%")
        else:
            print(str(list(a))+" -> "+str(list(b))+" = "+str(sab/sa*100)+"%")
            print(str(list(b))+" -> "+str(list(a))+" = "+str(sab/sb*100)+"%")
    curr = 1 # counter for that particular element
    print("Interesting rules:", end=' ') # Finding out the rules which have maximum support number among others 
    '''
    Finding out the rules which have maximum support number among others 
    '''
    for a in c:
        b = l-a
        ab = l
        sab = 0
        sa = 0
        sb = 0
        for q in data:
            temp = set(q)
            if(a.issubset(temp)):
                sa+=1
            if(b.issubset(temp)):
                sb+=1
            if(ab.issubset(temp)):
                sab+=1
        if(sa == 0):
            temp = 0
        else:
            temp = sab/sa*100
        if(temp == mmax):
            print(curr, end = ' ')
        curr += 1
        if(sb == 0):
            temp = 0
        else:
            temp = sab/sb*100
        if(temp == mmax):
            print(curr, end = ' ')
        curr += 1
    print()
    print()

['BREAD', 'CORNFLAKES', 'MILK'] -> ['BISCUIT'] = 100.0%
['BISCUIT'] -> ['BREAD', 'CORNFLAKES', 'MILK'] = 14.285714285714285%
['BREAD', 'MILK', 'BISCUIT'] -> ['CORNFLAKES'] = 50.0%
['CORNFLAKES'] -> ['BREAD', 'MILK', 'BISCUIT'] = 16.666666666666664%
['CORNFLAKES', 'MILK', 'BISCUIT'] -> ['BREAD'] = 100.0%
['BREAD'] -> ['CORNFLAKES', 'MILK', 'BISCUIT'] = 7.6923076923076925%
['BREAD', 'CORNFLAKES', 'BISCUIT'] -> ['MILK'] = 100.0%
['MILK'] -> ['BREAD', 'CORNFLAKES', 'BISCUIT'] = 20.0%
Interesting rules: 1 5 7 

['COFFEE', 'COCK', 'CORNFLAKES'] -> ['BISCUIT'] = 100.0%
['BISCUIT'] -> ['COFFEE', 'COCK', 'CORNFLAKES'] = 28.57142857142857%
['COFFEE', 'COCK', 'BISCUIT'] -> ['CORNFLAKES'] = 100.0%
['CORNFLAKES'] -> ['COFFEE', 'COCK', 'BISCUIT'] = 33.33333333333333%
['COFFEE', 'CORNFLAKES', 'BISCUIT'] -> ['COCK'] = 100.0%
['COCK'] -> ['COFFEE', 'CORNFLAKES', 'BISCUIT'] = 66.66666666666666%
['BISCUIT', 'COCK', 'CORNFLAKES'] -> ['COFFEE'] = 100.0%
['COFFEE'] -> ['BISCUIT', 'COCK', 'CORNFLAKES'] = 25.

# Fp Growth
references 

- http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpmax/ 

- https://towardsdatascience.com/fp-growth-frequent-pattern-generation-in-data-mining-with-python-implementation-244e561ab1c3 
-

In [31]:
import csv
from  csv import reader
from collections import defaultdict
from itertools import chain, combinations
import numpy as np
import pandas as pd
dataset = []
# with open('GroceryStore.csv', newline='') as f:
with open('GroceryStore.csv', newline='') as f:
    for ele in csv.reader(f):       
        dataset.append(list(ele[0].split(",")))

In [32]:

import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
con = TransactionEncoder()
con_arr = con.fit(dataset).transform(dataset)
df = pd.DataFrame(con_arr, columns = con.columns_)
df

,BISCUIT,BOURNVITA,BREAD,COCK,COFFEE,CORNFLAKES,JAM,MAGGI,MILK,SUGER,TEA
0,True,False,True,False,False,False,False,False,True,False,False
1,True,False,True,False,False,True,False,False,True,False,False
2,False,True,True,False,False,False,False,False,False,False,True
3,False,False,True,False,False,False,True,True,True,False,False
4,True,False,False,False,False,False,False,True,False,False,True
5,False,True,True,False,False,False,False,False,False,False,True
6,False,False,False,False,False,True,False,True,False,False,True
7,True,False,True,False,False,False,False,True,False,False,True
8,False,False,True,False,False,False,True,True,False,False,True
9,False,False,True,False,False,False,False,False,True,False,False


In [33]:
from mlxtend.frequent_patterns import apriori
apriori(df, min_support=0.6, use_colnames=True)

,support,itemsets
0,0.65,(BREAD)


In [34]:
%pip install mlxtend --upgrade

Referemce:
- http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/fpgrowth/
- 

In [35]:
%pip install fpgrowth_py

In [36]:
'''
Using Fp growth library to find out the all the sets of itemsets
as well as rules with their probabilities off association and 
choosing only the ones with which have maximum probability
'''
# from mlxtend.frequent_patterns import fpgrowth
from fpgrowth_py import fpgrowth
freqItemSet, rules = fpgrowth(dataset, minSupRatio=0.01, minConf=0.5)
# frequent_itemsets = fpgrowth(df, min_support=0.6, use_colnames=True)
#printing the dataset
print("ItemSet")
for ele in freqItemSet:
    print(ele)
print("Rules")
for ele in rules:
    if(ele[2] == 1):
        print(ele)

ItemSet
{'JAM'}
{'MILK', 'JAM'}
{'BREAD', 'MILK', 'JAM'}
{'MAGGI', 'MILK', 'JAM'}
{'BREAD', 'MAGGI', 'MILK', 'JAM'}
{'TEA', 'JAM'}
{'TEA', 'BREAD', 'JAM'}
{'TEA', 'MAGGI', 'JAM'}
{'TEA', 'BREAD', 'MAGGI', 'JAM'}
{'MAGGI', 'JAM'}
{'BREAD', 'JAM'}
{'BREAD', 'MAGGI', 'JAM'}
{'COCK'}
{'BREAD', 'COCK'}
{'BREAD', 'COFFEE', 'COCK'}
{'COCK', 'CORNFLAKES'}
{'COFFEE', 'COCK', 'CORNFLAKES'}
{'COCK', 'BISCUIT'}
{'COCK', 'CORNFLAKES', 'BISCUIT'}
{'COFFEE', 'COCK', 'BISCUIT'}
{'COFFEE', 'COCK', 'CORNFLAKES', 'BISCUIT'}
{'COFFEE', 'COCK'}
{'BOURNVITA'}
{'COFFEE', 'BOURNVITA'}
{'COFFEE', 'BOURNVITA', 'SUGER'}
{'TEA', 'BOURNVITA'}
{'TEA', 'BREAD', 'BOURNVITA'}
{'SUGER', 'BOURNVITA'}
{'BREAD', 'SUGER', 'BOURNVITA'}
{'BREAD', 'BOURNVITA'}
{'MILK'}
{'MAGGI', 'MILK'}
{'BREAD', 'MAGGI', 'MILK'}
{'TEA', 'MILK'}
{'TEA', 'CORNFLAKES', 'MILK'}
{'COFFEE', 'MILK'}
{'TEA', 'COFFEE', 'MILK'}
{'COFFEE', 'CORNFLAKES', 'MILK'}
{'TEA', 'COFFEE', 'CORNFLAKES', 'MILK'}
{'MILK', 'BISCUIT'}
{'CORNFLAKES', 'MILK', 'BISCUIT'

## On the Online Retail Dataset [Large Data set]

### About the dataset
- Each invoice has various *StockCode* per customer
- *StockCode* has a one-to-one mapping to the *Description*

In [37]:
df = pd.read_csv('OnlineRetail.csv',header=0)
# df['StockCode']
df.head

<bound method NDFrame.head of        InvoiceNo StockCode                          Description  Quantity  \
0         536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1         536365     71053                  WHITE METAL LANTERN         6   
2         536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3         536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4         536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   
...          ...       ...                                  ...       ...   
541904    581587     22613          PACK OF 20 SPACEBOY NAPKINS        12   
541905    581587     22899         CHILDREN'S APRON DOLLY GIRL          6   
541906    581587     23254        CHILDRENS CUTLERY DOLLY GIRL          4   
541907    581587     23255      CHILDRENS CUTLERY CIRCUS PARADE         4   
541908    581587     22138        BAKING SET 9 PIECE RETROSPOT          3   

             InvoiceDate  UnitPrice  Customer

### Cleaning the data
- removing spaces from some of the descriptions
- dropping the rows that don’t have invoice numbers 

https://www.kaggle.com/code/hassanamin/market-basket-analysis-for-online-retail-dataset/notebook

In [38]:
#Cleaning the data code
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)
df['InvoiceNo'] = df['InvoiceNo'].astype('str')

### Making a list of [*InvoiceNo*, [*different StockCode per invoice*]]

In [39]:
data =[]
for invoice_no in df['InvoiceNo'].unique():
    local_list = []
    local_list.append(invoice_no)
    df1 = df[df['InvoiceNo']== invoice_no]
    local_list.append(list(df1['StockCode'].values))
    data.append(local_list)
# mydic = {}
# for i in df['InvoiceNo']:
#     mydic[i].append(df.loc[df['InvoiceNo'] == 541908, 'StockCode'].iloc[1])
data      

[['536365',
  ['85123A', '71053', '84406B', '84029G', '84029E', '22752', '21730']],
 ['536366', ['22633', '22632']],
 ['536367',
  ['84879',
   '22745',
   '22748',
   '22749',
   '22310',
   '84969',
   '22623',
   '22622',
   '21754',
   '21755',
   '21777',
   '48187']],
 ['536368', ['22960', '22913', '22912', '22914']],
 ['536369', ['21756']],
 ['536370',
  ['22728',
   '22727',
   '22726',
   '21724',
   '21883',
   '10002',
   '21791',
   '21035',
   '22326',
   '22629',
   '22659',
   '22631',
   '22661',
   '21731',
   '22900',
   '21913',
   '22540',
   '22544',
   '22492',
   'POST']],
 ['536371', ['22086']],
 ['536372', ['22632', '22633']],
 ['536373',
  ['85123A',
   '71053',
   '84406B',
   '20679',
   '37370',
   '21871',
   '21071',
   '21068',
   '82483',
   '82486',
   '82482',
   '82494L',
   '84029G',
   '84029E',
   '22752',
   '21730']],
 ['536374', ['21258']],
 ['536375',
  ['85123A',
   '71053',
   '84406B',
   '20679',
   '37370',
   '21871',
   '21071',
   '210

#### [007] Defining the functions for utility
Functions to 
- writing to csv file
- returning the powerset 
- getting the data from a particular filename
- 
- get a union set of 2 sets
- pruning the dataset with less frequency
- returning list with check--> confidence > minimum confidence indicating the association rules

In [42]:
def dataToCSV(fname):
    first = True
    currentID = 1
    with open(fname, 'r') as dataFile, open(fname + '.csv', 'w') as outputCSV:
        for line in dataFile:
            nums = line.split()
            itemSetID = nums[1]
            item = nums[2]
            if(int(itemSetID) == currentID):
                if(first):
                    outputCSV.write(item)
                else:
                    outputCSV.write(',' + item)
                first = False
            else:
                outputCSV.write('\n' + item)
                currentID += 1

def powerset(s):
    return chain.from_iterable(combinations(s, r) for r in range(1, len(s)))

def getFromFile(fname):
    itemSets = []
    itemSet = set()

    with open(fname, 'r') as file:
        csv_reader = reader(file)
        for line in csv_reader:
            line = list(filter(None, line))
            record = set(line)
            for item in record:
                itemSet.add(frozenset([item]))
            itemSets.append(record)
    return itemSet, itemSets

def getAboveMinSup(itemSet, itemSetList, minSup, globalItemSetWithSup):
    freqItemSet = set()
    localItemSetWithSup = defaultdict(int)

    for item in itemSet:
        for itemSet in itemSetList:
            if item.issubset(itemSet):
                globalItemSetWithSup[item] += 1
                localItemSetWithSup[item] += 1

    for item, supCount in localItemSetWithSup.items():
        support = float(supCount / len(itemSetList))
        if(support >= minSup):
            freqItemSet.add(item)

    return freqItemSet


def getUnion(itemSet, length):
    return set([i.union(j) for i in itemSet for j in itemSet if len(i.union(j)) == length])


def pruning(candidateSet, prevFreqSet, length):
    tempCandidateSet = candidateSet.copy()
    for item in candidateSet:
        subsets = combinations(item, length)
        for subset in subsets:
            # if the subset is not in previous K-frequent get, then remove the set
            if(frozenset(subset) not in prevFreqSet):
                tempCandidateSet.remove(item)
                break
    return tempCandidateSet


def associationRule(freqItemSet, itemSetWithSup, minConf):
    rules = []
    for k, itemSet in freqItemSet.items():
        for item in itemSet:
            subsets = powerset(item)
            for s in subsets:
                confidence = float(
                    itemSetWithSup[item] / itemSetWithSup[frozenset(s)])
                if(confidence > minConf):
                    rules.append([set(s), set(item.difference(s)), confidence])
    return rules


def getItemSetFromList(itemSetList):
    tempItemSet = set()

    for itemSet in itemSetList:
        for item in itemSet:
            tempItemSet.add(frozenset([item]))
    return tempItemSet

In [43]:
from csv import reader
from collections import defaultdict
from itertools import chain, combinations
from optparse import OptionParser

def apriori(itemSetList, minSup, minConf):
    C1ItemSet = getItemSetFromList(itemSetList)
    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules

def aprioriFromFile(fname, minSup, minConf):
    C1ItemSet, itemSetList = getFromFile(fname)

    # Final result global frequent itemset
    globalFreqItemSet = dict()
    # Storing global itemset with support count
    globalItemSetWithSup = defaultdict(int)

    L1ItemSet = getAboveMinSup(
        C1ItemSet, itemSetList, minSup, globalItemSetWithSup)
    currentLSet = L1ItemSet
    k = 2

    # Calculating frequent item set
    while(currentLSet):
        # Storing frequent itemset
        globalFreqItemSet[k-1] = currentLSet
        # Self-joining Lk
        candidateSet = getUnion(currentLSet, k)
        # Perform subset testing and remove pruned supersets
        candidateSet = pruning(candidateSet, currentLSet, k-1)
        # Scanning itemSet for counting support
        currentLSet = getAboveMinSup(
            candidateSet, itemSetList, minSup, globalItemSetWithSup)
        k += 1

    rules = associationRule(globalFreqItemSet, globalItemSetWithSup, minConf)
    rules.sort(key=lambda x: x[2])

    return globalFreqItemSet, rules

# if __name__ == "__main__":
#     optparser = OptionParser()
#     optparser.add_option('-f', '--inputFile',
#                          dest='inputFile',
#                          help='CSV filename',
#                          default=None)
#     optparser.add_option('-s', '--minSupport',
#                          dest='minSup',
#                          help='Min support (float)',
#                          default=0.5,
#                          type='float')
#     optparser.add_option('-c', '--minConfidence',
#                          dest='minConf',
#                          help='Min confidence (float)',
#                          default=0.5,
#                          type='float')

#     (options, args) = optparser.parse_args()

# freqItemSet, rules = aprioriFromFile(options.inputFile, options.minSup, options.minConf)
freqItemSet, rules = aprioriFromFile('OnlineRetail1.csv', 0.7, 0.7)

KeyboardInterrupt: ignored

In [ ]:
print(freqItemSet)
print(rules)

In [ ]:
dataset = []
# with open('GroceryStore.csv', newline='') as f:
with open('OnlineRetail1.csv', newline='') as f:
    for ele in csv.reader(f):       
        dataset.append(list(ele[0].split(",")))

In [ ]:
con_arr = con.fit(dataset).transform(dataset)
df = pd.DataFrame(con_arr, columns = con.columns_)
df

In [ ]:
freqItemSet, rules = fpgrowth(dataset, minSupRatio=0.01, minConf=0.5)
# frequent_itemsets = fpgrowth(df, min_support=0.6, use_colnames=True)
#printing the dataset
print("ItemSet")
for ele in freqItemSet:
    print(ele)
print("Rules")
for ele in rules:
    if(ele[2] == 1):
        print(ele)